In [26]:
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np
import pyvista

from dolfinx.fem import Constant, Function, FunctionSpace, assemble_scalar, dirichletbc, form, locate_dofs_geometrical
from dolfinx.fem.petsc import assemble_matrix, assemble_vector, apply_lifting, create_vector, set_bc
from dolfinx.io import VTXWriter
from dolfinx.mesh import create_rectangle
from dolfinx.mesh import create_box
from dolfinx.plot import vtk_mesh
from ufl import (FacetNormal, FiniteElement, Identity, TestFunction, TrialFunction, VectorElement,
                 div, dot, ds, dx, inner, lhs, nabla_grad, rhs, sym)


"""GENERATE MESH AND TIME STEP"""

side = np.pi
mesh = create_rectangle(MPI.COMM_WORLD, [np.array([0.0, 0.0]), np.array([side, side])], [10, 10])

t = 0
T = 10
num_steps = 200
dt = T / num_steps

"""DEFINE THE FINITE ELEMENT FUNCTION SPACE"""

v_cg2 = VectorElement("Lagrange", mesh.ufl_cell(), 2)
s_cg1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
V = FunctionSpace(mesh, v_cg2)
Q = FunctionSpace(mesh, s_cg1)
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)

"""DEFINE BOUNDARY CONDITIONS"""


def bound_loop(dofs, f, xy): #x = 1, y = 0 mi dice quale delle due variabili voglio che sia nulla
    array_2d = np.zeros((dofs, 2), dtype=PETSc.ScalarType)
    for i in range(dofs):
        if xy == 1:
            array_2d[i][0] = 0
            array_2d[i][1] = f(i * side / dofs)
        else:
            array_2d[i][0] = f(i * side / dofs)
            array_2d[i][1] = 0
    return array_2d

def right_wall(x):
    return np.isclose(x[0], side)

right_wall_dofs = locate_dofs_geometrical(V, right_wall)
right_wall_D = bound_loop(len(right_wall_dofs), np.sin, 0)

right_bc = []
for i in range(len(right_wall_D)):
   right_wall_dofs_temp = np.array([  right_wall_dofs[i]  ],  dtype=np.int32)
   right_bc +=  [   dirichletbc(right_wall_D[i,0], right_wall_dofs_temp, Q)   ]

print(right_bc)






[<dolfinx.fem.bcs.DirichletBC object at 0x76d694d6cc10>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a77580>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a776a0>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a76350>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a777f0>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a76290>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a76020>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a750f0>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a761a0>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a75810>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a76200>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a74670>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a75e10>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a75d20>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a75cf0>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a75c60>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694a75ba0>, <dolfinx.fem.bcs.DirichletBC object at 0x76d694

TypeError: unsupported operand type(s) for +=: 'DirichletBC' and 'DirichletBC'

In [2]:
import numpy as np
print(type(right_wall_D[0]))
print(type(right_wall_dofs))
print(right_wall_dofs)
print(s_cg1)
print(right_wall)
arr = np.array([1, 2, 3], dtype=np.int32)
right_bc = dirichletbc( right_wall_D[0], arr, Q)
#c = [  1   2   3  12  13  25  26  42  43  63  64  88  89 117 118 150 151 187
 #188 228 229]
print(right_bc)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[2]
<CG1 on a triangle>
<function right_wall at 0x76d6955f4670>


RuntimeError: Rank mis-match between Constant and function space in DirichletBC

In [2]:
print(right_wall_dofs1)

1


In [9]:
print( right_wall_D[1,0] )
print(right_wall_dofs[0])
print(Q)
print(type(  right_wall_D[1,0] ))
print(right_wall_D[1,0])
print(right_wall_dofs)

0.14904226617617444
1
FunctionSpace(Mesh(blocked element (Basix element (P, triangle, 1, gll_warped, unset, False), (2,)), 0), FiniteElement('Lagrange', triangle, 1))
<class 'numpy.float64'>
0.14904226617617444
[  1   2   3  12  13  25  26  42  43  63  64  88  89 117 118 150 151 187
 188 228 229]


In [39]:
def bound_loop(dofs, f, xy): #x = 1, y = 0 mi dice quale delle due variabili voglio che sia nulla
    array_2d = np.zeros((dofs, 2), dtype=PETSc.ScalarType)
    for i in range(dofs):
        if xy == 1:
            array_2d[i][0] = 0
            array_2d[i][1] = f(i * side / dofs)
        else:
            array_2d[i][0] = f(i * side / dofs)
            array_2d[i][1] = 0
    return array_2d

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. dolfinx.cpp.fem.DirichletBC_float64(g: numpy.ndarray[numpy.float64], dofs: numpy.ndarray[numpy.int32], V: dolfinx::fem::FunctionSpace<double>)
    2. dolfinx.cpp.fem.DirichletBC_float64(g: dolfinx::fem::Constant<double>, dofs: numpy.ndarray[numpy.int32], V: dolfinx::fem::FunctionSpace<double>)
    3. dolfinx.cpp.fem.DirichletBC_float64(g: dolfinx::fem::Function<double, double>, dofs: numpy.ndarray[numpy.int32])
    4. dolfinx.cpp.fem.DirichletBC_float64(g: dolfinx::fem::Function<double, double>, dofs: List[numpy.ndarray[numpy.int32][2]], V: dolfinx::fem::FunctionSpace<double>)

Invoked with: array(2.), [2], <dolfinx.cpp.fem.FunctionSpace_float64 object at 0x7d49983fdd70>

<class 'int'>
2
<class 'int'>
